# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [43]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, IDF, StringIndexer
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.sql.functions import udf, concat, col, lit
from pyspark.sql.types import IntegerType

import re

# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 

In [62]:
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

In [72]:
df.take(1)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

### Build Body Length Feature

In [5]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [6]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [7]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [11]:
# TODO: write your code to answer question 1
df.filter(df.Id == "1112").select("BodyLength").show()

+----------+
|BodyLength|
+----------+
|        63|
+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [12]:
df.columns

['Body', 'Id', 'Tags', 'Title', 'oneTag', 'words', 'BodyLength']

In [34]:
# TODO: write your code to answer question 2
df.createOrReplaceTempView("oneTagSmall")
df_title_body = spark.sql("""
    SELECT
        Id,
        CONCAT(title, body) AS title_body
    FROM
        oneTagSmall
""")

In [35]:
regexTokenizer = RegexTokenizer(inputCol="title_body", outputCol="words", pattern="\\W")
df_title_body = regexTokenizer.transform(df_title_body)

In [40]:
df_title_body = df_title_body.withColumn("title_body_length", body_length(df_title_body.words))

In [41]:
df_title_body.filter(df_title_body.Id == "5123").select("title_body_length").show()

+-----------------+
|title_body_length|
+-----------------+
|              135|
+-----------------+



# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [44]:
# TODO: write your code to create this vector
assembler = VectorAssembler(inputCols=["title_body_length"], outputCol="NumFeatures")
df_title_body = assembler.transform(df_title_body)

In [45]:
df_title_body.take(1)

[Row(Id=1, title_body="How to check if an uploaded file is an image without mime type?<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", words=['how', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'without', 'mime', 'type', 'p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'typ

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [46]:
# TODO: write your code to answer question 3
scaler = Normalizer(inputCol="NumFeatures", outputCol="ScaledNumFeatures")
df_title_body = scaler.transform(df_title_body)

In [47]:
df_title_body.filter(df_title_body.Id == "512").select(col("id"), col("ScaledNumFeatures")).show()

+-----------------+
|ScaledNumFeatures|
+-----------------+
|            [1.0]|
+-----------------+



# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [51]:
# TODO: write your code to answer question 4
standard_scaler = StandardScaler(inputCol="NumFeatures", 
                                 outputCol="StandardScaledNumFeatures", 
                                 withMean=True, 
                                 withStd=True)
scalerModel = standard_scaler.fit(df_title_body)
df_title_body = scalerModel.transform(df_title_body)

In [52]:
df_title_body.filter(df_title_body.Id == "512").select(col("id"), col("StandardScaledNumFeatures")).show()

+---+-------------------------+
| id|StandardScaledNumFeatures|
+---+-------------------------+
|512|     [-0.6417314460998...|
+---+-------------------------+



# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [59]:
# TODO: write your code to answer question 5
min_max_scaler = MinMaxScaler(inputCol="NumFeatures", 
                                 outputCol="MinMaxScaledNumFeatures")
min_max_model = min_max_scaler.fit(df_title_body)
df_title_body = min_max_model.transform(df_title_body)

In [60]:
df_title_body.filter(df_title_body.Id == "512").select(col("id"), col("MinMaxScaledNumFeatures")).show()

+---+-----------------------+
| id|MinMaxScaledNumFeatures|
+---+-----------------------+
|512|   [0.00624833820792...|
+---+-----------------------+



# Linear Regression

In [104]:
data = spark.sql("""
    SELECT
        *,
        CONCAT(title, body) AS title_body
    FROM
        oneTagSmall
""")

In [105]:
regexTokenizer = RegexTokenizer(inputCol="title_body", outputCol="tb_words", pattern="\\W")
data = regexTokenizer.transform(data)

In [106]:
data = data.withColumn("title_body_length", body_length(data.tb_words))

In [107]:
assembler = VectorAssembler(inputCols=["title_body_length"], outputCol="NumFeatures")
data = assembler.transform(data)

In [108]:
regexTokenizer = RegexTokenizer(inputCol="Tags", outputCol="tags_words", pattern="\\W")
data = regexTokenizer.transform(data)

In [109]:
data = data.withColumn("num_tags", body_length(data.tags_words))

In [110]:
assembler = VectorAssembler(inputCols=["num_tags"], outputCol="NumTagsFeatures")
data = assembler.transform(data)

In [111]:
lr = LinearRegression(maxIter=5, regParam=0.0, fitIntercept=False, solver="normal")

In [114]:
train_data = data.select(
    col("num_tags").alias("label"), 
    col("NumFeatures").alias("features")
)

In [115]:
lrModel = lr.fit(train_data)

In [116]:
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.010406137876]
Intercept: 0.0
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
|   7.001010763906576|
| 0.13629055629422737|
| -28.966644791082985|
|  1.7096389033793276|
|  3.3974547671001325|
|  3.2195396593020127|
|   0.740857317007247|
|   4.230956561084562|
|  2.8865432472708714|
|   4.229945797177986|
|   1.491110007983891|
|  2.4598915943559714|
|   2.750252690976644|
|  2.4900992440773146|
|   1.500505381953288|
|-0.04798763375356074|
| -0.9543323928698024|
| -0.2477257812102036|
|  3.7200450412553008|
|   3.795920298106841|
+--------------------+
only showing top 20 rows

RMSE: 3.244695
r2: 0.416533


In [118]:
data.createOrReplaceTempView("tempview")
spark.sql("""
    select 
        min(title_body_length), 
        max(title_body_length),
        max(title_body_length)/min(title_body_length)
    from
        tempview
""").show()

+----------------------+----------------------+---------------------------------------------------------------------------------+
|min(title_body_length)|max(title_body_length)|(CAST(max(title_body_length) AS DOUBLE) / CAST(min(title_body_length) AS DOUBLE))|
+----------------------+----------------------+---------------------------------------------------------------------------------+
|                    10|                  7532|                                                                            753.2|
+----------------------+----------------------+---------------------------------------------------------------------------------+



In [ ]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

df_stats = data.select(
    _mean(col('title_body_length')).alias('mean'),
    _stddev(col('title_body_length')).alias('std')
).collect()

In [121]:
print(df_stats[0]["mean"], df_stats[0]["std"])

180.28187 192.10819533505023
